# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from random import sample

import re

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV

from timeit import default_timer as timer

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

In [5]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9707,HOBBIES_2_145_CA_4_validation,HOBBIES_2_145,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9708,HOBBIES_2_146_CA_4_validation,HOBBIES_2_146,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9709,HOBBIES_2_147_CA_4_validation,HOBBIES_2_147,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9710,HOBBIES_2_148_CA_4_validation,HOBBIES_2_148,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
INPUT_DIR_2 = r'/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/List of Product ID according to 4 demand patterns/California/Hobbies'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [ ]:
len(sales_intermittent)

1301

In [7]:
len(sales_lumpy)

231

In [ ]:
len(sales_erratic)

458

In [ ]:
len(sales_smooth)

3594

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [8]:
sales_lumpy

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
64,HOBBIES_1_066_CA_1_validation,HOBBIES_1_066,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,4,0,0,0,0,0,0,6,0
126,HOBBIES_1_131_CA_1_validation,HOBBIES_1_131,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,2,0,0,0,1,0,0,1,0
135,HOBBIES_1_140_CA_1_validation,HOBBIES_1_140,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,4,0,0,0,0,0,1,0,4
140,HOBBIES_1_145_CA_1_validation,HOBBIES_1_145,HOBBIES_1,HOBBIES,CA_1,CA,0,12,0,4,...,0,11,0,0,1,3,0,12,0,2
162,HOBBIES_1_167_CA_1_validation,HOBBIES_1_167,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9703,HOBBIES_2_141_CA_4_validation,HOBBIES_2_141,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9704,HOBBIES_2_142_CA_4_validation,HOBBIES_2_142,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9709,HOBBIES_2_147_CA_4_validation,HOBBIES_2_147,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9710,HOBBIES_2_148_CA_4_validation,HOBBIES_2_148,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# User-defined functions to calculate Metrics

In [9]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

# Format sales data and then merge with calendar data

In [10]:
sales_pattern = sales_lumpy.copy()

In [11]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

In [12]:
dataframe_weekly

,wm_yr_wk,HOBBIES_1_002_CA_2,HOBBIES_1_017_CA_4,HOBBIES_1_027_CA_4,HOBBIES_1_042_CA_2,HOBBIES_1_053_CA_3,HOBBIES_1_053_CA_4,HOBBIES_1_066_CA_1,HOBBIES_1_066_CA_2,HOBBIES_1_066_CA_3,...,HOBBIES_2_145_CA_3,HOBBIES_2_147_CA_4,HOBBIES_2_148_CA_1,HOBBIES_2_148_CA_2,HOBBIES_2_148_CA_3,HOBBIES_2_148_CA_4,HOBBIES_2_149_CA_1,HOBBIES_2_149_CA_2,HOBBIES_2_149_CA_3,HOBBIES_2_149_CA_4
0,11101,2.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,1.0
1,11102,1.0,1.0,0.0,2.0,0.0,6.0,0.0,0.0,0.0,...,5.0,0.0,1.0,1.0,1.0,1.0,3.0,6.0,0.0,7.0
2,11103,2.0,4.0,0.0,0.0,0.0,6.0,1.0,0.0,3.0,...,0.0,0.0,0.0,7.0,2.0,0.0,1.0,1.0,0.0,1.0
3,11104,1.0,2.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0,...,1.0,0.0,2.0,1.0,4.0,1.0,1.0,0.0,3.0,0.0
4,11105,0.0,0.0,0.0,2.0,6.0,1.0,0.0,0.0,1.0,...,0.0,0.0,3.0,4.0,4.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,1.0,5.0,0.0,2.0,0.0,0.0,3.0,0.0,1.0,...,0.0,0.0,2.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
269,11609,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,1.0,0.0,0.0,0.0,1.0
270,11610,1.0,1.0,0.0,2.0,0.0,4.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,2.0,1.0
271,11611,2.0,2.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0


In [13]:
lags = list(range(1, 25))
moving_averages = list(range(1, 25)) 
stds = list(range(2, 25))
maximums = list(range(1, 25))
minimums = list(range(1, 25))
totals = list(range(1, 25))
num_days_zeros = list(range(1, 25))

correlation_threshold = 0.8

end_train_week = 11610

num_id_sample_for_tuning = 200

In [14]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

In [15]:
dataframe_check = dataframe_weekly[sample_list].iloc[-2:].T

dataframe_check['Check'] = np.where((dataframe_check[271] == 0) &
                                    (dataframe_check[272] == 0), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()

In [16]:
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [17]:
len(sample_list_filtered)

123

In [ ]:
start = timer()

df_output_hyper_params = pd.DataFrame()
    
for product in sample_list_filtered:
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    dataframe_product = dataframe_weekly[['wm_yr_wk', product]]

    # Create Lag variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (t-{lag})': dataframe_product[product].shift(lag)
        for lag in lags
    })

    # Create Rolling Moving Average variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
        for moving_average in moving_averages
    })

    # Create Rolling Standard Deviation variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
        for std in stds
    })

    # Create Rolling Maximum variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
        for maximum in maximums
    })

    # Create Rolling Minimum variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
        for minimum in minimums
    })

    # Create Rolling Total variables 
    dataframe_product = dataframe_product.assign(**{
        f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
        for total in totals
    })


    dataframe_product.dropna(inplace=True)
    dataframe_product.set_index('wm_yr_wk', inplace=True)

    train_data = dataframe_product[dataframe_product.index <= end_train_week]

    corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
    high_corr_var=np.where(corr_matrix > correlation_threshold)
    high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]

    train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)

    X_train = train_data.drop(product, axis=1)
    y_train = train_data[product]  

    tscv = TimeSeriesSplit(n_splits=3)

    params = {'n_estimators': [30, 50],
              'max_depth': list(range(3,11)),
              'min_samples_leaf': range(4,11),
              'max_features': ['sqrt', 'log2', None],
              'bootstrap': [True], #We force bootstrap
              'max_samples': [0.6, 0.7, 0.8, 0.9]}

    forest = RandomForestRegressor(n_jobs=1)

    regressor = RandomizedSearchCV(estimator=forest,
                                  param_distributions=params,
                                  cv = tscv, 
                                  scoring='neg_mean_absolute_percentage_error',
                                  n_iter=150,
                                  verbose=1)
    regressor.fit(X_train, y_train)
    best_params_temp = regressor.best_params_

    df_output_temp = pd.DataFrame.from_dict(best_params_temp, orient='index').T
    df_output_temp['Product'] = product

    df_output_hyper_params = df_output_hyper_params.append(df_output_temp, ignore_index=True)

    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: FOODS_3_138_CA_2
Progressing: 0.0 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_2_334_CA_3
Progressing: 0.85 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_030_CA_1
Progressing: 1.69 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_763_CA_3
Progressing: 2.54 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_1_123_CA_1
Progressing: 3.39 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_608_CA_2
Progressing: 4.24 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_1_010_CA_2
Progressing: 5.08 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_120_CA_4
Progressing: 5.93 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_784_CA_1
Progressing: 

In [ ]:
df_output_hyper_params.to_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/Random Forest (Completed Pipeline)/Foods/Lumpy_Params_Tuning_Random Forest.csv')

# Decide optimum set of parameters for Random Forest algorithm

In [ ]:
# See if any values larger than 1 in column "Product"
# If yes, the corresponding set of parameters occur more than once and those sets should be considered first
df_output_hyper_params.groupby(['n_estimators', 'max_depth', 'min_samples_leaf', 'max_features', 'max_samples'])[['Product']].count().sort_values(by = 'Product', ascending=False)

Product
n_estimators max_depth min_samples_leaf max_features max_samples         
30           3         5                sqrt         0.8                1
             4         10               sqrt         0.6                1
             6         9                log2         0.6                1
             7         4                log2         0.9                1
                       9                log2         0.7                1
             8         6                log2         0.6                1
                       10               sqrt         0.6                1
             9         4                log2         0.7                1
                                                     0.8                1
                       5                log2         0.7                1
                                        sqrt         0.7                1
                       9                sqrt         0.6                1
             10        9                log2         0.8                1
50           8         4                log2         0.9                1

In [ ]:
df_output_hyper_params.groupby(['n_estimators'])[['Product']].count()

,Product
n_estimators,
30,80
50,38


In [ ]:
df_output_hyper_params.groupby(['max_depth'])[['Product']].count()

,Product
max_depth,
3,16
4,18
5,11
6,20
7,18
8,12
9,15
10,8


In [ ]:
df_output_hyper_params.groupby(['min_samples_leaf'])[['Product']].count()

,Product
min_samples_leaf,
4,22
5,18
6,10
7,17
8,14
9,17
10,20


In [ ]:

df_output_hyper_params.groupby(['max_features'])[['Product']].count()

,Product
max_features,
log2,9
sqrt,5


In [ ]:
df_output_hyper_params.groupby(['max_samples'])[['Product']].count()

,Product
max_samples,
0.6,24
0.7,27
0.8,32
0.9,35


# Implement Random Forest on Test Set with optimum set of parameters

### Specify optimum set of parameters

In [18]:
n_estimators_chosen = 30
max_depth_chosen = 6
min_samples_leaf_chosen = 4
max_features_chosen = 'log2'
max_samples_chosen = 0.9

In [19]:
len(list_pattern_py)

231

In [20]:
list_pattern_temp = list_pattern_py.copy()

In [21]:
start = timer()

df_output_rf_pattern = pd.DataFrame()
for product in list_pattern_temp:
  print('Currently Running: %s' % product)
  print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))
  dataframe_product = dataframe_weekly[['wm_yr_wk', product]]
  # Create Lag variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (t-{lag})': dataframe_product[product].shift(lag)
      for lag in lags
  })

  # Create Rolling Moving Average variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
      for moving_average in moving_averages
  })

  # Create Rolling Standard Deviation variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
      for std in stds
  })

  # Create Rolling Maximum variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
      for maximum in maximums
  })

  # Create Rolling Minimum variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
      for minimum in minimums
  })

  # Create Rolling Total variables 
  dataframe_product = dataframe_product.assign(**{
      f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
      for total in totals
  })

  dataframe_product.dropna(inplace=True)
  dataframe_product.set_index('wm_yr_wk', inplace=True)

  train_data = dataframe_product[dataframe_product.index <= end_train_week]
  test_data = dataframe_product[dataframe_product.index > end_train_week]

  corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
  high_corr_var=np.where(corr_matrix > correlation_threshold)
  high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]

  train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
  test_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)

  X_train = train_data.drop(product, axis=1)
  y_train = train_data[product]

  X_test = test_data.drop(product, axis=1)
  y_test = test_data[product]

  forest = RandomForestRegressor(n_jobs=1,
                                n_estimators = n_estimators_chosen,
                                min_samples_leaf = min_samples_leaf_chosen,
                                max_samples = max_samples_chosen,
                                max_features = max_features_chosen,
                                max_depth = max_depth_chosen,
                                bootstrap = [True],
                                random_state = 0)
  

  forest.fit(X_train,y_train)
  y_pred = forest.predict(X_test)
  df_output_temp = pd.DataFrame({'Actual Data': y_test, 
                                  'Forecast': y_pred, 
                                  'Product': [product for num_rows in range(len(y_pred))]
                                  })
  
  df_output_rf_pattern = df_output_rf_pattern.append(df_output_temp, ignore_index=False)  

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: HOBBIES_1_066_CA_1
Progressing: 0.0 %
Currently Running: HOBBIES_1_131_CA_1
Progressing: 0.43 %
Currently Running: HOBBIES_1_140_CA_1
Progressing: 0.87 %
Currently Running: HOBBIES_1_145_CA_1
Progressing: 1.3 %
Currently Running: HOBBIES_1_167_CA_1
Progressing: 1.73 %
Currently Running: HOBBIES_1_223_CA_1
Progressing: 2.16 %
Currently Running: HOBBIES_1_234_CA_1
Progressing: 2.6 %
Currently Running: HOBBIES_1_267_CA_1
Progressing: 3.03 %
Currently Running: HOBBIES_1_299_CA_1
Progressing: 3.46 %
Currently Running: HOBBIES_1_318_CA_1
Progressing: 3.9 %
Currently Running: HOBBIES_1_331_CA_1
Progressing: 4.33 %
Currently Running: HOBBIES_1_401_CA_1
Progressing: 4.76 %
Currently Running: HOBBIES_2_002_CA_1
Progressing: 5.19 %
Currently Running: HOBBIES_2_007_CA_1
Progressing: 5.63 %
Currently Running: HOBBIES_2_011_CA_1
Progressing: 6.06 %
Currently Running: HOBBIES_2_014_CA_1
Progressing: 6.49 %
Currently Running: HOBBIES_2_017_CA_1
Progressing: 6.93 %
Currently Running:

In [22]:
df_output_rf_pattern

,Actual Data,Forecast,Product
wm_yr_wk,,,
11611,0.0,1.496665,HOBBIES_1_066_CA_1
11612,3.0,1.225165,HOBBIES_1_066_CA_1
11611,20.0,8.034251,HOBBIES_1_131_CA_1
11612,8.0,8.360363,HOBBIES_1_131_CA_1
11611,0.0,2.227305,HOBBIES_1_140_CA_1
...,...,...,...
11612,0.0,0.049772,HOBBIES_2_147_CA_4
11611,0.0,1.113317,HOBBIES_2_148_CA_4
11612,0.0,0.439103,HOBBIES_2_148_CA_4


In [23]:
df_output_rf_pattern.to_csv('Lumpy_Random_Forest_Forecast_Values.csv')

In [24]:
# User-defined funcion to calculate metrics

def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [25]:
# Start calculating metrics 
start = timer()

df_forecast_metrics = df_output_rf_pattern.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.00709013698333365 minutes


In [26]:
df_forecast_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_002_CA_2,0.509385,101.8770,137.9248,54.9934,1.018770,1.021993
1,HOBBIES_1_017_CA_4,0.649800,129.9599,126.6505,42.0851,1.299599,1.377907
2,HOBBIES_1_027_CA_4,inf,inf,200.0000,NaN,0.022052,0.022052
3,HOBBIES_1_042_CA_2,inf,32.1354,38.3101,32.1354,0.642708,0.643548
4,HOBBIES_1_053_CA_3,inf,inf,200.0000,NaN,1.292050,1.305265
...,...,...,...,...,...,...,...
226,HOBBIES_2_148_CA_4,inf,inf,200.0000,NaN,0.776210,0.846253
227,HOBBIES_2_149_CA_1,0.860476,57.3651,71.7625,46.6596,0.860476,1.118797
228,HOBBIES_2_149_CA_2,inf,inf,200.0000,NaN,1.322531,1.342045
229,HOBBIES_2_149_CA_3,0.498494,99.6987,112.8708,22.8063,0.498494,0.567123


In [27]:
df_forecast_metrics.to_csv('Random_Forest_Lumpy_Forecast_Metrics.csv')